## 1. 라이브러리 및 데이터 불러오기

In [1]:
import pandas as pd

In [164]:
kosdaq = pd.read_excel("C:/Users/hanah/Desktop/workspace/Project/ESG/crawling/dart/kosdaq_report_final.xlsx")
ind_cd = pd.read_excel('한국표준산업분류(10차).xlsx')

## 2. 기업개요 중 업종코드 전처리

In [171]:
company_info = kosdaq.iloc[:,:13]
company_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1388 entries, 0 to 1387
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   고유번호     1388 non-null   int64         
 1   사업자등록번호  1388 non-null   object        
 2   종목코드     1388 non-null   int64         
 3   회사명      1388 non-null   object        
 4   업종       1388 non-null   object        
 5   주요제품     1385 non-null   object        
 6   상장일      1388 non-null   datetime64[ns]
 7   대표자명     1388 non-null   object        
 8   홈페이지     1324 non-null   object        
 9   지역       1388 non-null   object        
 10  주소       1388 non-null   object        
 11  업종코드     1388 non-null   int64         
 12  설립일      1388 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(8)
memory usage: 141.1+ KB


In [159]:
company_info['업종코드'] = company_info['업종코드'].astype('str')
company_info['ind_code'] = company_info['업종코드'].str[:2]

In [160]:
ind_dict = dict([(str(idx), value) for idx, value in zip(ind_cd['코드'], ind_cd['항목명'])])
for idx, code in enumerate(company_info['ind_code']):
    company_info.loc[idx, 'ind_code'] = ind_dict[code]

In [161]:
company_info['ind_code'].value_counts()

제조업                         918
정보통신업                       205
도매 및 소매업                     96
전문, 과학 및 기술 서비스업             70
금융 및 보험업                     39
건설업                          28
사업시설 관리, 사업 지원 및 임대 서비스업     12
교육 서비스업                       8
운수 및 창고업                      6
예술, 스포츠 및 여가관련 서비스업           5
숙박 및 음식점업                     3
Name: ind_code, dtype: int64

In [162]:
company_info.drop(['고유번호', '업종', '홈페이지', '지역', '업종코드', '상장일'], axis=1, inplace=True)
company_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1390 entries, 0 to 1389
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   사업자등록번호   1390 non-null   object
 1   종목코드      1390 non-null   int64 
 2   회사명       1390 non-null   object
 3   주요제품      1387 non-null   object
 4   대표자명      1390 non-null   object
 5   주소        1390 non-null   object
 6   설립일       1390 non-null   int64 
 7   ind_code  1390 non-null   object
dtypes: int64(2), object(6)
memory usage: 87.0+ KB


In [163]:
company_info.to_excel('company_info.xlsx', index=False)

## 3. 공시정보 전처리

In [198]:
temp1 = kosdaq.iloc[:,13:29]
temp1.insert(0, '결산연도', 2021)
temp1.columns = ['결산연도', '최대주주_성명', '최대주주_관계', '최대주주_주식종류', '최대주주_기초주식수', '최대주주_기초주식지분율', 
                 '최대주주_기말주식수', '최대주주_기말주식지분율', '소액주주수', '주주수', '소액주주비율', '소액주주_주식수',
                 '총주식수', '소액주주_주식지분율', '감사의견', '감사의견_강조사항', '핵심감사사항']

temp2 = kosdaq.iloc[:,29:]
temp2.insert(0, '결산연도', 2020)
temp2.columns = temp1.columns 
temp = pd.concat([temp1, temp2],ignore_index=True)

In [201]:
dart = pd.concat([kosdaq['회사명'], temp], axis=1)
dart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2776 entries, 0 to 2775
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   회사명           1388 non-null   object
 1   결산연도          2776 non-null   int64 
 2   최대주주_성명       2770 non-null   object
 3   최대주주_관계       2770 non-null   object
 4   최대주주_주식종류     2770 non-null   object
 5   최대주주_기초주식수    2770 non-null   object
 6   최대주주_기초주식지분율  2770 non-null   object
 7   최대주주_기말주식수    2770 non-null   object
 8   최대주주_기말주식지분율  2770 non-null   object
 9   소액주주수         2770 non-null   object
 10  주주수           2770 non-null   object
 11  소액주주비율        2770 non-null   object
 12  소액주주_주식수      2770 non-null   object
 13  총주식수          2770 non-null   object
 14  소액주주_주식지분율    2770 non-null   object
 15  감사의견          2770 non-null   object
 16  감사의견_강조사항     2770 non-null   object
 17  핵심감사사항        2770 non-null   object
dtypes: int64(1), object(17)
memory usage: 390.5+ KB


In [202]:
dart.to_excel('dart_info.xlsx', index=False)